In [1]:
import typing
import io
import os

import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json

from urllib.request import urlretrieve

from PIL import Image
from torchvision import transforms

from models.modeling_seg import ActiveTestVisionTransformer, CONFIGS

from utils.data_utils_feature import FeatureDataset, tensor_ordinal_to_float

def read_one_results(path):
    with open(path, "r") as outfile:
        data = json.load(outfile)
    return data

def write_one_results(json_data, path):
    with open(path, "w") as outfile:
        json.dump(json_data, outfile)

In [2]:
%%capture
config = CONFIGS["ViT-B_16"]
config.input_feature_dim = 25
model = ActiveTestVisionTransformer(config, num_classes=50, zero_head=False, img_size=480, vis=True)
model_checkpoint = "./output/ViT-output-PSPNet-VOC-train-ordinal-new_checkpoint_3600.bin"
model.load_state_dict(torch.load(model_checkpoint))
model.eval()

In [3]:
split = "val"
model_data_path = "/workspace/pytorch-segmentation/pro_data/PSPNet_VOC/"
annotation_dir = model_data_path + split + "/image_true_losses.npy"
inputs_path = model_data_path + split
train_loader = FeatureDataset(inputs_path, annotation_dir, aug=False)

In [9]:
estimate_loss_list = None
for i in range(1449):
    feature, label = train_loader.__getitem__(i)
    patch_loss_estimation = model.forward_patch_result(feature.unsqueeze(0))
    estimated_loss = tensor_ordinal_to_float(patch_loss_estimation)
    estimated_loss = estimated_loss.numpy()
    if estimate_loss_list is None:
        estimate_loss_list = estimated_loss
    else:
        estimate_loss_list = np.concatenate((estimate_loss_list, estimated_loss))
    if (i+1)%100 == 0:
        print(f"Process {i}")

Process 99
Process 199
Process 299
Process 399
Process 499
Process 599
Process 699
Process 799
Process 899
Process 999
Process 1099
Process 1199
Process 1299
Process 1399


In [13]:
store_path = "./output/region_16_16/ViT-output-PSPNet-VOC-train-ordinal-new_losses_3600.json"
json_object = {"losses":estimate_loss_list.tolist()}
write_one_results(json_object, store_path)

In [4]:
feature, label = train_loader.__getitem__(0)
patch_loss_estimation = model.forward_patch_result(feature.unsqueeze(0))
estimated_loss = tensor_ordinal_to_float(patch_loss_estimation)
estimated_loss = estimated_loss.numpy()

In [8]:
estimated_loss.mean()

0.02411111